In [6]:
import openai 
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np

load_dotenv()

# API Configuration
openai.api_key = "sk-None-puO4ccikA7Bd612BIdj2T3BlbkFJqUjtZO8q63FaoXMUYgKC"

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
from nemoguardrails import RailsConfig, LLMRails

In [9]:
# read user data from SKY API

df = pd.read_csv("data/user_data.csv")
df.tail()

,ID,Name,Title,First Name,Last Name,Gender,Fundraiser Status,Address,City,State,Postal Code,Country,Phone,Phone Type,Email,Email Type,Date Added,Date Modified,Deceased
495,46623,Charlotte Anderson,Mrs.,Charlotte,Anderson,Female,NaN,2273 Caminito Preciosa Norta,La Jolla,CA,92037-7231,United States,(858) 272-9549,Home,NaN,NaN,1998-10-28T08:28:33-04:00,2024-12-05T15:37:00.786-05:00,False
496,328841,Christie Anderson,Mrs.,Christie,Anderson,Female,NaN,7220 Calle Cristobal Unit 14,San Diego,CA,92126-6055,United States,(858) 414-2273,Phone,chr15t13a113n@hotmail.com,Email,2021-11-19T12:50:52.25-05:00,2024-11-23T22:01:56.772-05:00,False
497,326707,Christine Anderson,NaN,Christine,Anderson,Female,NaN,402 Lupine Way,Oceanside,CA,92057-8544,United States,(760) 390-2833,Cell,cband42@gmail.com,Email,2021-08-20T01:45:16.147-04:00,2024-09-09T17:28:48.942-04:00,False
498,269480,Darlene Anderson,Ms.,Darlene,Anderson,Female,NaN,2428 C St Apt B,San Diego,CA,92102-8001,United States,(619) 282-4571,Home,NaN,NaN,2008-10-20T16:47:45.387-04:00,2024-06-11T14:06:24.506-04:00,False
499,32824,David Anderson,Mr.,David,Anderson,Unknown,NaN,11358 Pegasus Ave,San Diego,CA,92126-1546,United States,(858) 693-3583,Home,NaN,NaN,1998-10-28T08:16:52-04:00,2024-09-09T17:39:34.851-04:00,False
